# ETL_EDA
This file records the process of acquiring raw data, traforming them, and loading them into a MongoDB. The data are store (almost) in their raw form.

## 1. Raw Data from BPA 
The [dataset](https://transmission.bpa.gov/business/operations/Wind/baltwg.txt) is a continuously updated txt file in csv format. It contains the energy producation and load in the last 5 days. There will be some blank lines at the end for today. It can be retrieved simply by making `requests` without parameters. For more complicated API call, you may need to add query parameters.

In [1]:
import requests

url = "https://transmission.bpa.gov/business/operations/Wind/baltwg.txt"
req = requests.get(url, timeout=0.5)
req.raise_for_status()
text = req.text
print(text[:1000])
print('...')
print(text[-500:])

BPA Balancing Authority Load & Total Wind Generation
at 5-minute intervals, last 7 days
Dates: 08Nov2020 - 15Nov2020 (last updated 14Nov2020 20:11:01) Pacific Time
Based on 5-min MW readings from the BPA SCADA system for points 45583, 79687, 79682, 164377, 70681

This represents loads and resources in BPA's Balancing Authority (BA) including some that are not BPA's.
It does not include BPA loads served by transfer, scheduled out of region,
or scheduled to customers with their own BAs such as Seattle and Tacoma

BPA/Technical Operations (TOT-OpInfo@bpa.gov)

Date/Time       	Load	Wind	Hydro	Fossil/Biomass	Nuclear
11/08/2020 00:00	5719	306	6249	545	1163
11/08/2020 00:05	5725	321	6300	583	1161
11/08/2020 00:10	5732	361	6237	638	1164
11/08/2020 00:15	5705	403	6195	681	1169
11/08/2020 00:20	5704	442	6156	692	1169
11/08/2020 00:25	5698	484	6097	714	1163
11/08/2020 00:30	5670	504	5999	729	1168
11/08/2020 00:35	5679	531	5903	784	1169
11/08/2020 00:40	5667	543	5802	831	1164
...
2020 22:10					


## 2. Raw Data to Documents/Dicts
Using pandas, it is simple to parse a in-memory string. The first few lines of description need to be skipped. Datetime conversion is made and blank lines are dropped. Now the data can be easily converted to a list of dicts which is what we want for the MongoDB.

In [2]:
import pandas
from io import StringIO

df = pandas.read_csv(StringIO(text), skiprows=11, delimiter='\t')
df.columns = df.columns.str.strip()             # remove space in columns name
df['Datetime'] = pandas.to_datetime(df['Date/Time'])
df.drop(columns=['Date/Time'], axis=1, inplace=True)
df.dropna(inplace=True)  
df.head()

,Load,Wind,Hydro,Fossil/Biomass,Nuclear,Datetime
0,5719.0,306.0,6249.0,545.0,1163.0,2020-11-08 00:00:00
1,5725.0,321.0,6300.0,583.0,1161.0,2020-11-08 00:05:00
2,5732.0,361.0,6237.0,638.0,1164.0,2020-11-08 00:10:00
3,5705.0,403.0,6195.0,681.0,1169.0,2020-11-08 00:15:00
4,5704.0,442.0,6156.0,692.0,1169.0,2020-11-08 00:20:00


## 3. Upsert MongoDB
If we fetch the data frequently, there are lots of duplicate data entry between each run. The de-duplication happens at insertion. The MongoDB API to use is `collection.replace_one(filter=..., replacement=..., upsert=True)`. The statement matches a document in MongoDB with `filter`, replaces it with `replacement` if the document exists or inserts `replacement` into the database if `filter` matches nothing. 

In [3]:
import pymongo

client = pymongo.MongoClient()

In [4]:
db = client.get_database("energy")
collection = db.get_collection("energy")
update_count = 0
for record in df.to_dict('records'):
    result = collection.replace_one(
        filter={'Datetime': record['Datetime']},    # locate the document if exists
        replacement=record,                         # latest document
        upsert=True)                                # update if exists, insert if not
    if result.matched_count > 0:
        update_count += 1
print(f"rows={df.shape[0]}, update={update_count}, "
      f"insert={df.shape[0]-update_count}")

rows=1971, update=1971, insert=0
